<a href="https://colab.research.google.com/github/Dicere/WB_Internship/blob/main/0_2_FE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install pymorphy2
!pip string
!pip install emoji
!pip install catboost
!pip install xgboost
!pip install lightgbm
!pip install pyspellchecker
!pip install deeppavlov
!pip install transformers

In [ ]:
!pip install lazypredict

In [2]:
import nltk
import string
import pymorphy2
nltk.download('stopwords')
nltk.download('punkt')
nltk.download("wordnet")
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
import collections
import re
from spellchecker import SpellChecker
from nltk.sentiment import SentimentIntensityAnalyzer
from deeppavlov import build_model, configs
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizerFast
import torch
from transformers import AutoModelForSequenceClassification
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from catboost import CatBoostClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from lazypredict.Supervised import LazyClassifier

In [21]:
from sklearn.metrics import roc_auc_score, f1_score, classification_report, recall_score, precision_recall_curve,precision_score, precision_recall_fscore_support

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment', return_dict=True)
morph = pymorphy2.MorphAnalyzer()
tqdm.pandas()

In [5]:
df = pd.read_csv('/content/drive/MyDrive/WB_стажировка/wb_school_task_2.csv.gzip',compression='gzip')
df = df.drop_duplicates(ignore_index=True)

In [6]:
stop_words = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()
em= ['☝️', '☹', '☺️', '✌', '✨', '❤', '❤️', '❤️\u200d🔥', '🌸', '🌹', '🌺', '🌼', '🎁', '🎄', '🐈\u200d⬛', '🐤', '🐰', '👌', '👌🏻', '👌🏼', '👍', '👍🏻', '👍🏼', '👍🏽', '👍🏿', '👎🏼', '👏', '👾', '💄', '💋', '💐', '💓', '💕', '💖', '💗', '💘', '💙', '💛', '💜', '💞', '💣', '💥', '💩', '💪', '💫', '💯', '🔥', '😀', '😁', '😂', '😃', '😅', '😉', '😊', '😋', '😌', '😍', '😏', '😒', '😔', '😘', '😜', '😞', '😡', '😢', '😣', '😭', '😼', '😿', '🙂', '🙈', '🙌', '🙌🏻', '🙍', '🙏', '🛁', '🤍', '🤔', '🤗', '🤙', '🤣', '🤤', '🤦\u200d♀️', '🤩', '🤪', '🤬', '🤮', '🤷\u200d♀️', '🥰', '🥲', '🥳', '🥶', '🥺', '🪔', '🫒', '\U0001fae7', '\U0001faf6🏻']
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([char for char in text if char not in em])
    tokens = text.split()
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in stop_words]
    return ' '.join(tokens)

df['text_preproc'] = df['text'].apply(preprocess_text)
text = ' '.join(df[df['label']==0].text_preproc)

In [7]:
pred_obr_text = {'part_of_speech':['NOUN','ADJF','ADJS','COMP','VERB','INFN','PRTF','PRTS','GRND','NUMR','ADVB','NPRO','PRED','PREP','CONJ','PRCL','INTJ'],
                 'case_s':['nomn','gent','datv','accs','ablt','loct','voct','gen2','acc2','loc2'],
                 'forms_of_plurality':['sing','plur'],
                 'genus_category':['masc','femn','neut']}
spell = SpellChecker(language='ru')
#  # Инициализация объекта лемматизатора
# morph = pymorphy2.MorphAnalyzer(lang='ru')

def generate_feat(x):
# Define set of stop words
    stop_words = set(stopwords.words('russian'))

    # Function to count word frequencies
    def count_word_frequencies(text):
        words = text.split()
        return Counter(words)

    # Function to count words in a sentence
    def count_words(sentence):
        return len(sentence.split())

    # Function to check if a text contains a certain word
    def contains_word(text, word):
        return int(word in text.lower())

    # Function to count character frequencies
    def count_char_frequencies(text):
        chars = list(text)
        return Counter(chars)

    # Function to check if a text contains a certain character
    def contains_char(text, char):
        return int(char in text)
    
    def punct(text):
        punctuation = string.punctuation.replace("!", "").replace(".", "").replace("?", "")
        for char in text:
          if char in punctuation:
              return True
        return False
    
    def count_spelling_errors(sentence):

      sentence = re.sub(r'[^\w\s-]', '', sentence)
      words = sentence.split()
      error_count = 0

      for word in words:

        if type(word)==str:
          corrected_word = spell.correction(word)
          if word != corrected_word:
            error_count += 1
      # Возвращаем количество ошибок
      return error_count


    def pos_tag(x):
      words = x.text.split()
      pos_tags = []
      part_of_speech_list = []
      case_s_list = []
      forms_of_plurality_list =[]
      genus_category_list=[]
      for word in words:
          word = word.lower().strip('.,!?-()[]{}:;\'"')
          parsed_word = morph.parse(word)[0]

          part_of_speech_list.append(parsed_word.tag.POS)
          case_s_list.append(parsed_word.tag.case)
          forms_of_plurality_list.append(parsed_word.tag.number)
          genus_category_list.append(parsed_word.tag.gender)

      part_of_speech_count = collections.Counter(part_of_speech_list)
      case_s_count = collections.Counter(case_s_list)
      forms_of_plurality_count = collections.Counter(forms_of_plurality_list)
      genus_category_count = collections.Counter(genus_category_list)

      dict_list = {"part_of_speech":part_of_speech_count,"case_s":case_s_count,"forms_of_plurality":forms_of_plurality_count,"genus_category":genus_category_count}

      for i in list(pred_obr_text.keys()):
        for n in pred_obr_text[i]:
          if n in dict_list[i].keys():
            df[n]= dict_list[i][n]
          else:
            df[n]= 0
      return df


    # Generate features
    df['contains_,'] = df['text'].progress_apply(lambda x: int(" ," in x))
    df['contains_double_space'] = df['text'].progress_apply(lambda x: int("  " in x))
    df['contains_punctuation'] = df['text'].progress_apply(lambda x: int(punct(x)))
    df['text_length'] = df['text'].progress_apply(len)
    df['num_sentences'] = df['text'].progress_apply(lambda x: len(nltk.sent_tokenize(x)))
    df['num_words'] = df['text'].progress_apply(lambda x: len(nltk.word_tokenize(x)))
    df['count_error'] = df['text'].progress_apply(lambda x: count_spelling_errors(x))
    df['percent_error_word'] = df['count_error'] / df['num_words']
    df['percent_error_sentence_true'] = df['count_error'] / df['num_sentences']
    df['num_stopwords'] = df['text'].progress_apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))
    df['percent_stopwords'] = df['num_stopwords'] / df['num_words']
    df['num_punctuation'] = df['text'].progress_apply(lambda x: len([char for char in x if char in string.punctuation]))
    df['num_unique_words'] = df['text'].progress_apply(lambda x: len(set(x.split())))
    df['num_uppercase'] = df['text'].progress_apply(lambda x: len([char for char in x if char.isupper()]))
    df['num_exclamation_marks'] = df['text'].progress_apply(lambda x: x.count('!'))
    df['num_question_marks'] = df['text'].progress_apply(lambda x: x.count('?'))
    df['avg_word_length'] = df['text'].progress_apply(lambda x: sum(len(word) for word in x.split()) / len(x.split()))
    df['avg_words_per_sentence'] = df['text'].progress_apply(lambda x: sum(count_words(sentence) for sentence in nltk.sent_tokenize(x)) / len(nltk.sent_tokenize(x)))
    df['contains_@'] = df['text'].progress_apply(lambda x: contains_char(x, '@'))
    df['contains_#'] = df['text'].progress_apply(lambda x: contains_char(x, '#'))
    b = df.progress_apply(lambda x: pos_tagging(x),axis=1)
    return df 

In [8]:
pred_obr_text = {'part_of_speech':['NOUN','ADJF','ADJS','COMP','VERB','INFN','PRTF','PRTS','GRND','NUMR','ADVB','NPRO','PRED','PREP','CONJ','PRCL','INTJ'],
                 'case_s':['nomn','gent','datv','accs','ablt','loct','voct','gen2','acc2','loc2'],
                 'forms_of_plurality':['sing','plur'],
                 'genus_category':['masc','femn','neut']}

morph = pymorphy2.MorphAnalyzer()
def pos_tagging(x):
    words = x.text.split()
    pos_tags = []
    part_of_speech_list = []
    case_s_list = []
    forms_of_plurality_list =[]
    genus_category_list=[]
    for word in words:
        word = word.lower().strip('.,!?-()[]{}:;\'"')
        parsed_word = morph.parse(word)[0]

        part_of_speech_list.append(parsed_word.tag.POS)
        case_s_list.append(parsed_word.tag.case)
        forms_of_plurality_list.append(parsed_word.tag.number)
        genus_category_list.append(parsed_word.tag.gender)

    part_of_speech_count = collections.Counter(part_of_speech_list)
    case_s_count = collections.Counter(case_s_list)
    forms_of_plurality_count = collections.Counter(forms_of_plurality_list)
    genus_category_count = collections.Counter(genus_category_list)
    
    dict_list = {"part_of_speech":part_of_speech_count,"case_s":case_s_count,"forms_of_plurality":forms_of_plurality_count,"genus_category":genus_category_count}
    return dict_list

In [9]:
emoji_dict = {
    "positive": ["☺️", "✨", "❤", "❤️", "❤️\u200d🔥", "🌸", "🌹", "🌺", "🌼", "🎁", "🎄", "🐤", "🐰", "👌", "👌🏻", "👌🏼", "👍", "👍🏻", "👍🏼", "👍🏽", "👍🏿", "👏", "💄", "💋", "💐", "💓", "💕", "💖", "💗", "💘", "💙", "💛", "💜", "💞", "💫", "💯", "🔥", "😀", "😁", "😂", "😃", "😅", "😉", "😊", "😋", "😌", "😍", "😘", "😜", "🙂", "🙌", "🙌🏻", "🙏", "🛁", "🤍", "🤗", "🤙", "🤣", "🤩", "🤪", "🥰", "🥳", "🪔", "🫒", "\U0001fae7", "\U0001faf6🏻"],
    "neutral": ["☝️", "✌", "🐈\u200d⬛", "👾", "💣", "💥", "💩", "💪", "😉", "😏", "😼", "🙈", "🙍", "🤔", "🥲", "🥶", "🥺"],
    "negative": ["☹", "😒", "😔", "😞", "😡", "😢", "😣", "😭", "😿", "🤤", "🤦\u200d♀️", "🤬", "🤮", "🤷\u200d♀️"]
}


def generate_feat_emoji_ton(x):
  
    def tone(x):
      pridicted =[0,0,0]
      if type(x)==str:
        inputs = tokenizer(x, padding=True, truncation=True, return_tensors="pt")
        outputs = model(**inputs)
        predicted = torch.nn.functional.softmax(outputs.logits, dim=1).detach().numpy()[0]
      return list(predicted)
    
    def emo_sentiment(x):
      em = [e['emoji'] for e in emoji.emoji_list(x)]
      emojes=['positive','neutral','negative']
      for i in em:
        for key, val in emoji_dict.items():
          if i in val:
            emojes.append(key)
      dicts = collections.Counter(emojes)
      new_dict = {key: value - 1 for key, value in dicts.items()}
      return new_dict


    def has_multiple_exclamation_marks(text):
      return bool(re.search(r'!!+', text))

    b = df.progress_apply(lambda x: pos_tagging(x),axis=1)

    for i in pred_obr_text.keys():
      for n in pred_obr_text[i]:
        j = []
        for m in range(len(b)):
          if n in b[m][i].keys():
            j.append(b[m][i][n])
          else:
            j.append(b[m][i][n])
        df[n]=j
    df['count_emoji'] = df['text'].progress_apply(lambda x: len(emoji.distinct_emoji_list(x)))
    df['emoji_per_word'] = df['count_emoji'] / df['num_words']
    df['emoji_per_sentence'] = df['count_emoji'] / df['num_sentences']
    df['contains_!_and_more'] = df['text'].progress_apply(lambda x: int(has_multiple_exclamation_marks(x)))
    df[['neutral','positive','negative']] = df['text'].progress_apply(lambda x: pd.Series(tone(x)))
    df[['positive_emoj','neutral_emoj','negative_emoj']] = df['text'].progress_apply(lambda x: pd.Series(emo_sentiment(x)))
    
    return df 

In [ ]:
pipeline = (df.pipe(generate_feat)
            .pipe(generate_feat_emoji_ton))

In [17]:
cat_feat = ['text']
feature=pipeline[pipeline.columns[3:]].columns

In [25]:
X = pipeline[feature].drop(columns=['label','text','text_preproc'])
y = pipeline['label']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [27]:
def predict_precsion_recall(y_test, pred):
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  return [precision, recall]

In [28]:
clf = LazyClassifier(verbose=0,ignore_warnings=False, custom_metric=predict_precsion_recall)
models, predictions = clf.fit(X_train[X_train.columns[1:]], X_test[X_test.columns[1:]], y_train, y_test)
models[['Precision', 'Recall']] = models['predict_precsion_recall'].apply(lambda x: pd.Series(x))
models.drop(columns=['predict_precsion_recall'],inplace=True)
models.sort_values(by='F1 Score', ascending=False,inplace=True)
models

 21%|██        | 6/29 [00:08<00:27,  1.19s/it]

CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X)


 90%|████████▉ | 26/29 [00:13<00:00,  3.70it/s]

StackingClassifier model failed to execute
StackingClassifier.__init__() missing 1 required positional argument: 'estimators'


100%|██████████| 29/29 [00:14<00:00,  1.95it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken,Precision,Recall
Model,,,,,,,
BaggingClassifier,0.76,0.59,0.59,0.72,1.32,0.62,0.24
XGBClassifier,0.74,0.59,0.59,0.71,1.27,0.52,0.28
LGBMClassifier,0.75,0.59,0.59,0.71,0.35,0.55,0.25
AdaBoostClassifier,0.74,0.57,0.57,0.70,1.25,0.50,0.22
RandomForestClassifier,0.76,0.56,0.56,0.69,0.79,0.71,0.15
KNeighborsClassifier,0.73,0.56,0.56,0.69,0.09,0.46,0.20
NuSVC,0.76,0.55,0.55,0.68,0.55,0.81,0.10
LogisticRegression,0.74,0.55,0.55,0.68,0.10,0.50,0.15
RidgeClassifier,0.74,0.55,0.55,0.68,0.05,0.51,0.14


In [29]:
clf = CatBoostClassifier(verbose=0)
clf.fit(X_train, y_train)

In [ ]:
X_test

In [30]:
threshold = 0.5

In [31]:
y_pred = clf.predict_proba( X_test)[:, 1]
y_pred_metki = (y_pred> threshold).astype('int')

In [32]:
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_metki, average='macro')
roc = roc_auc_score(y_test, y_pred)
print("AUC_ROC----"+str(roc))
print("F1-score-----"+str(f1))
print("Precision-----"+str(precision))
print("Recall-----"+str(recall))

AUC_ROC----0.7644005270092228
F1-score-----0.6110696252044663
Precision-----0.7577846681620266
Recall-----0.6029644268774703


###Бейзлайн без новых фичей
##AUC_ROC----0.7663504611330698
##F1-score-----0.6477303317854499
##Precision-----0.7589015743926597
##Recall-----0.6308629776021081